### Baselines:
- BERT-based classifier trained on the data

### Ideas:
- Using NLI model: https://huggingface.co/microsoft/deberta-base-mnli and fine-tuning it on the data we have.
- Validity on the premise (sentence) level.

In [1]:
import json
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', None)

In [6]:
from datasets import load_dataset, load_metric, Dataset

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

import torch

In [2]:
from sklearn.metrics import precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    return {
        'recall' : recall,
        'precision': precision,
        'f1': f1,
        
    }

In [3]:
nli_tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-base-mnli')
nli_model     = AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-base-mnli')
arg_stance_pipeline = TextClassificationPipeline(model=nli_model, tokenizer=nli_tokenizer, framework='pt', task='validity_classifier', device=0)

Some weights of the model checkpoint at microsoft/deberta-base-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
taska_training_df = pd.read_csv('../data/TaskA_train.csv')
taskb_training_df = pd.read_csv('../data/TaskB_train.csv')

taska_valid_df = pd.read_csv('../data/TaskA_dev.csv')
taskb_valid_df = pd.read_csv('../data/TaskB_dev.csv')

In [34]:
taska_training_df = taska_training_df[taska_training_df.Validity != 0]
taska_training_df['input_txt'] = taska_training_df.apply(lambda x: '[CLS] {} [SEP] {} [SEP]'.format(x['Premise'], x['Conclusion']), axis=1)
resutls = arg_stance_pipeline(taska_training_df['input_txt'].tolist())
taska_training_df['pred_label'] = [1 if x['label'] == 'ENTAILMENT' else -1 for x in resutls]

In [35]:
taska_training_df[['topic', 'Validity', 'pred_label']].head(n=10)

,topic,Validity,pred_label
0,TV viewing is harmful to children,1,-1
1,TV viewing is harmful to children,1,-1
2,TV viewing is harmful to children,1,-1
3,TV viewing is harmful to children,1,1
4,TV viewing is harmful to children,-1,-1
5,TV viewing is harmful to children,1,1
6,TV viewing is harmful to children,1,1
7,TV viewing is harmful to children,1,1
8,TV viewing is harmful to children,-1,1
9,TV viewing is harmful to children,-1,-1


In [36]:
taska_training_df.Validity.value_counts()

 1    401
-1    320
Name: Validity, dtype: int64

In [38]:
precision, recall, f1, _ = precision_recall_fscore_support(taska_training_df.Validity.tolist(), taska_training_df.pred_label.tolist(), average='binary')

print('Precision: {}, Recall {}, F1: {}'.format(precision, recall, f1))

Precision: 0.8571428571428571, Recall 0.47880299251870323, F1: 0.6144


#### Fine-tune the NLI model on the training data:

#### Fine-tune simple BERT model on the taining data: